In [127]:
import tensorflow as tf
from   tensorflow import keras
from   tensorflow.keras import regularizers
from   tensorflow.keras import Sequential
from   tensorflow.keras.layers import Dropout, Dense

from   IPython import display
from   matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile

import config

In [128]:
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [129]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

In [130]:
TICKER = 'SPY'
EXPIRY = '2020-08-21'

In [131]:
data = np.load(os.path.join(config.ML_DATA_DIR, '{}_{}.npz'.format(TICKER, EXPIRY)))
X_train = data['train_examples']
Y_train = data['train_labels']
X_test = data['test_examples']
Y_test = data['test_labels']

In [132]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (data['train_examples'], data['train_labels']))
test_dataset = tf.data.Dataset.from_tensor_slices(
    (data['test_examples'], data['test_labels']))

In [133]:
BATCH_SIZE = 512
BUFFER_SIZE = 100
STEPS_PER_EPOCH = X_train.shape[0]//BATCH_SIZE

In [134]:
train_dataset = train_dataset.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).repeat()
validate_dataset = test_dataset.batch(BATCH_SIZE)

In [135]:
lr_schedule = keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    decay_steps=STEPS_PER_EPOCH*10,
    decay_rate=1,
    staircase=False
)

def get_optimizer():
    return keras.optimizers.Adam(lr_schedule)

In [147]:
checkpoint_filepath = os.path.join(config.ML_MODELS_DIR, 'checkpoint')
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)

def get_callbacks(name):
    return [
        # tfdocs.modeling.EpochDots(),
        model_checkpoint_callback,
        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3),
        tf.keras.callbacks.TensorBoard(logdir/name),
    ]

In [137]:
def compile_and_fit(model, name, optimizer=None, max_epochs=100):
    if optimizer is None:
        optimizer = get_optimizer()
    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[
            tf.keras.losses.BinaryCrossentropy(
                from_logits=True, name='binary_crossentropy'),
            'accuracy'
        ]
    )

    model.summary()

    history = model.fit(
        train_dataset,
        steps_per_epoch = STEPS_PER_EPOCH,
        epochs=max_epochs,
        validation_data=validate_dataset,
        callbacks=get_callbacks(name),
        verbose=1)
    return history

In [138]:
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

In [139]:
size_histories = {}

In [140]:
size_histories['test'] = compile_and_fit(model, 'sizes/test')

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_85 (Dense)             (None, 256)               4864      
_________________________________________________________________
dense_86 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_87 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_88 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_89 (Dense)             (None, 1)                 33        
Total params: 48,129
Trainable params: 48,129
Non-trainable params: 0
_________________________________________________________________
Train for 1188 steps, validate for 510 steps
Epoch 1/20
1188/1188 [==============================] - 41s 34ms/step 

In [151]:
#docs_infra: no_execute

# Load the TensorBoard notebook extension
%load_ext tensorboard

# Open an embedded TensorBoard viewer
%tensorboard --logdir {logdir}/sizes

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 13990), started 0:01:01 ago. (Use '!kill 13990' to kill it.)